In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read All IPL Data
deliveres = pd.read_csv("C:/Users/utkar/OneDrive/Desktop/Analytics/Data/IPL Ball-by-Ball 2008-2020.csv")
matches = pd.read_csv("C:/Users/utkar/OneDrive/Desktop/Analytics/Data/IPL Matches 2008-2020.csv")

In [3]:
# Goal 
# Get Best Batsman against any team x, at venue y, during phase z 
# For example - Who are the best batsmen against 'Mumbai Indians' at 'Wankhede Stadium' during 'End' overs
# I've used MCDM with Topsis to arrive at a score for each batsman
# The Criterion considered are Strike Rate, Balls Per Dismissal, Runs Per Innings and Dot Percentage
# A random(not really !!) weight is assigned to each of the above. 
# We've narrowed the dataset to atleast 2 innings and IPL 2016 onwards

In [4]:
# Return match phase, based on over information
def get_phase(over): 
    if (over < 6) : 
        return 'PowerPlay'
    elif (over < 15) : 
        return 'Middle'
    else:
        return 'End'

In [5]:
# Balls per dismissal
def get_BPD(balls, dismissals): 
    if (dismissals == 0): 
        return balls/1
    else: 
        return balls/dismissals

In [6]:
# Balls per boundary
def get_BPB(balls, boundries): 
    if (boundries == 0): 
        return balls/1
    else: 
        return balls/boundries

In [7]:
# Create a mega DataFrame for all batsmen, 
# adding all extra information about RPI, SR, BPB, BPD and DP
# inputs are venue, opposition and match phase
def getCustom(df, venue, opposition, phase): 
    
    df = df[df.venue  == venue]
    df = df[df.bowling_team == opposition]
    df = df[df.phase == phase]
    
    df.reset_index(inplace = True, drop = True)
    
    df['isDot']   = df['batsman_runs'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne']   = df['batsman_runs'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo']   = df['batsman_runs'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['batsman_runs'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour']  = df['batsman_runs'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix']   = df['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)

    runs = df.groupby('batsman').batsman_runs.sum().reset_index().rename(columns = {'batsman_runs' : 'runs'})
    balls = df.groupby('batsman').ball.count().reset_index().rename(columns = {'ball' : 'balls'})
    innings = df.groupby('batsman').id.nunique().reset_index().rename(columns = {'id' : 'innings'})
    dismissals = df.groupby('batsman').player_dismissed.count().reset_index().rename(columns = {'player_dismissed' : 'dismissals'})
    sixes = df.groupby('batsman').isSix.sum().reset_index().rename(columns = {'isSix' : 'sixes'}) 
    fours = df.groupby('batsman').isFour.sum().reset_index().rename(columns = {'isFour' : 'fours'})
    dots = df.groupby('batsman').isDot.sum().reset_index().rename(columns = {'isDot' : 'dots'})
   
    mega = pd.merge(runs, balls, on = 'batsman', how = 'left').merge(innings, on = 'batsman').merge(dismissals, on = 'batsman').merge(fours, on = 'batsman').merge(sixes , on = 'batsman').merge(dots, on = 'batsman')
    mega['RPI'] = mega['runs'] / mega['innings']
    mega['SR'] = 100 * (mega['runs'] / mega['balls'])
    mega['BPB'] = mega.apply(lambda x: get_BPB(x['balls'], x['fours'] + x['sixes']) , axis=1)
    mega['BPD'] = mega.apply(lambda x: get_BPD(x['balls'], x['dismissals']) , axis=1)
    mega['DP'] = mega.apply(lambda x: 100 * (x['dots'] / x['balls']) , axis=1)
    
    return mega

In [8]:
def getTopsisScore(mydf, wt_sr, wt_rpi, wt_bpd, wt_dp):
    # Square the required fields and store in another column
    mydf['calc_SR']  = mydf['SR'].apply(lambda x: x * x)
    mydf['calc_RPI'] = mydf['RPI'].apply(lambda x: x * x)
    mydf['calc_BPD']  = mydf['BPD'].apply(lambda x: x * x)
    mydf['calc_DP']  = mydf['DP'].apply(lambda x: x * x)

    # get SQRT of sum of squares
    sr_sqrt , rpi_sqrt , bpd_sqrt , dp_sqrt = np.sqrt(mydf[['calc_SR' , 'calc_RPI' , 'calc_BPD' , 'calc_DP']].sum(axis=0))

    #divide calculated squares with sum of squares
    mydf['calc_SR'] = mydf['calc_SR'].apply(lambda x : x / sr_sqrt)
    mydf['calc_RPI'] = mydf['calc_RPI'].apply(lambda x : x / rpi_sqrt)
    mydf['calc_BPD'] = mydf['calc_BPD'].apply(lambda x : x / bpd_sqrt)
    mydf['calc_DP'] = mydf['calc_DP'].apply(lambda x : x / dp_sqrt)

    # multiply each calculated value with it's weight 
    mydf['calc_SR'] = mydf['calc_SR'].apply(lambda x : wt_sr * x)
    mydf['calc_RPI'] = mydf['calc_RPI'].apply(lambda x : wt_rpi * x)
    mydf['calc_BPD'] = mydf['calc_BPD'].apply(lambda x : wt_bpd * x)
    mydf['calc_DP'] = mydf['calc_DP'].apply(lambda x : wt_dp * x)

    # Get best and worst of each criterion
    best_sr , worst_sr   = max(mydf['calc_SR']) , min(mydf['calc_SR'])
    best_rpi , worst_rpi = max(mydf['calc_RPI']) , min(mydf['calc_RPI'])
    best_bpd , worst_bpd = max(mydf['calc_BPD']) , min(mydf['calc_BPD'])
    best_dp , worst_dp   = min(mydf['calc_DP']) , max(mydf['calc_DP'])
    
    # Deviation from best - Square of difference 
    mydf['dev_best_sr']   = mydf['calc_SR'].apply(lambda x : (x-best_sr) * (x-best_sr))
    mydf['dev_best_rpi']  = mydf['calc_RPI'].apply(lambda x : (x-best_rpi) * (x-best_rpi))
    mydf['dev_best_bpd']  = mydf['calc_BPD'].apply(lambda x : (x-best_bpd) * (x-best_bpd))
    mydf['dev_best_dp']   = mydf['calc_DP'].apply(lambda x : (x-best_dp) * (x-best_dp))

    # SquareRoot of sum of all deviations from best
    mydf['dev_best_sqrt'] = np.sqrt(mydf.apply(lambda x: x['dev_best_sr'] + x['dev_best_rpi'] + x['dev_best_bpd'] + x['dev_best_dp'] , axis=1))
    
    # Deviation from Worst - Square of difference 
    mydf['dev_worst_sr']   = mydf['calc_SR'].apply(lambda x : (x-worst_sr) * (x-worst_sr))
    mydf['dev_worst_rpi']  = mydf['calc_RPI'].apply(lambda x : (x-worst_rpi) * (x-worst_rpi))
    mydf['dev_worst_bpd']  = mydf['calc_BPD'].apply(lambda x : (x-worst_bpd) * (x-worst_bpd))
    mydf['dev_worst_dp']   = mydf['calc_DP'].apply(lambda x : (x-worst_dp) * (x-worst_dp))

    # SquareRoot of sum of all deviations from worst
    mydf['dev_worst_sqrt'] = np.sqrt(mydf.apply(lambda x: x['dev_worst_sr'] + x['dev_worst_rpi'] + x['dev_worst_bpd'] + x['dev_worst_dp'] , axis=1))
    
    # Calculate Score - worst / (worst + best) .. shows how far from worst are you (i.e - closer to best)
    mydf['score'] = mydf.apply(lambda x : x['dev_worst_sqrt'] / (x['dev_worst_sqrt'] + x['dev_best_sqrt']) , axis=1)
    
    return mydf

In [9]:
# Main - Start

# Create Copies
del_df = deliveres.copy()
match_df = matches.copy()

In [10]:
# Merge two dataframes
comb = pd.merge(del_df , match_df, on = 'id', how = 'left') 

# Get Data from IPL 2016 onwards
comb = comb[(comb.id > 980901)]

In [11]:
# debug
comb.head(1)

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,...,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
122652,980903,2,1,7,RV Uthappa,G Gambhir,Z Khan,4,0,4,...,Delhi Daredevils,Kolkata Knight Riders,field,Kolkata Knight Riders,wickets,9.0,N,NaN,S Ravi,C Shamshuddin


In [12]:
# Add Match Phase
comb['phase'] = comb['over'].apply(lambda x : get_phase(x))

In [13]:
# debug 
comb['bowling_team'].unique()

array(['Delhi Daredevils', 'Kolkata Knight Riders', 'Gujarat Lions',
       'Kings XI Punjab', 'Sunrisers Hyderabad',
       'Royal Challengers Bangalore', 'Mumbai Indians',
       'Rising Pune Supergiants', 'Rising Pune Supergiant',
       'Chennai Super Kings', 'Rajasthan Royals', 'Delhi Capitals'],
      dtype=object)

In [14]:
# debug
comb['venue'].unique()

array(['Eden Gardens',
       'Punjab Cricket Association IS Bindra Stadium, Mohali',
       'M Chinnaswamy Stadium', 'Saurashtra Cricket Association Stadium',
       'Feroz Shah Kotla', 'Rajiv Gandhi International Stadium, Uppal',
       'Wankhede Stadium', 'Maharashtra Cricket Association Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Green Park', 'Shaheed Veer Narayan Singh International Stadium',
       'Holkar Cricket Stadium', 'MA Chidambaram Stadium, Chepauk',
       'Sawai Mansingh Stadium', 'Sheikh Zayed Stadium',
       'Dubai International Cricket Stadium', 'Sharjah Cricket Stadium'],
      dtype=object)

In [15]:
# Get per batsman DF for stadium, oppostion, phase data
mydf = getCustom(comb , 'M Chinnaswamy Stadium' , 'Royal Challengers Bangalore' , 'Middle')

In [16]:
# debug
mydf.head(1)

,batsman,runs,balls,innings,dismissals,fours,sixes,dots,RPI,SR,BPB,BPD,DP
0,A Ashish Reddy,31,16,1,0,2,3,6,31.0,193.75,3.2,16.0,37.5


In [17]:
# Atleast 2 innings
mydf = mydf[(mydf.innings > 1)]

In [18]:
# Assumed weights (total out of 1)
wt_sr = 0.2
wt_rpi = 0.4
wt_bpd = 0.2
wt_dp = 0.2

In [19]:
final_df = getTopsisScore(mydf, wt_sr, wt_rpi, wt_bpd, wt_dp)

In [20]:
# debug 
# Sorted Result for venue = 'M. Chinnaswamy Staidum' , bowling_team = 'RCB' and phase = 'Middle'
# weights for each criterion are mentioned above
final_df[['batsman','runs' , 'SR' , 'RPI' , 'BPD' , 'DP' , 'innings', 'score']].sort_values(by = 'score', ascending = False)

,batsman,runs,SR,RPI,BPD,DP,innings,score
36,KS Williamson,104,182.456140,52.000000,57.0,26.315789,2,0.819816
65,SS Iyer,78,156.000000,39.000000,50.0,26.000000,2,0.590928
52,Q de Kock,56,193.103448,28.000000,29.0,17.241379,2,0.578848
15,DA Warner,57,183.870968,28.500000,15.5,29.032258,2,0.541994
73,Yuvraj Singh,55,171.875000,27.500000,32.0,34.375000,2,0.514069
4,AJ Finch,76,133.333333,38.000000,57.0,42.105263,2,0.507146
68,SW Billings,13,185.714286,6.500000,3.5,28.571429,2,0.483299
58,RV Uthappa,57,154.054054,28.500000,18.5,43.243243,2,0.436438
44,MS Dhoni,89,115.584416,22.250000,77.0,44.155844,4,0.425072
7,AT Rayudu,77,105.479452,25.666667,73.0,36.986301,3,0.410397
